In [2]:
import pandas as pd
import joblib
import io
from keras.models import load_model
import ipywidgets as widgets
from IPython.display import display, clear_output

# Upload widgets
model_upload = widgets.FileUpload(accept='.keras', multiple=False, description='Upload Model (.keras)')
scaler_upload = widgets.FileUpload(accept='.pkl', multiple=False, description='Upload Scaler (.pkl)')
feature_upload = widgets.FileUpload(accept='.pkl', multiple=False, description='Upload Features (.pkl)')
data_upload = widgets.FileUpload(accept='.csv', multiple=False, description='Upload Data (.csv)')
run_button = widgets.Button(description="Run Model", button_style='success')
output = widgets.Output()

# Global holders
model = None
scaler = None
feature_columns = None

# File saving function
def save_upload(upload_widget, filename):
    content = list(upload_widget.value.values())[0]['content']
    with open(filename, "wb") as f:
        f.write(content)

# Load uploaded files
def load_all_files():
    global model, scaler, feature_columns

    try:
        save_upload(model_upload, 'uploaded_model.keras')
        model = load_model('uploaded_model.keras')
    except Exception as e:
        with output:
            print(f"❌ Error loading model: {e}")
        return False

    try:
        save_upload(scaler_upload, 'uploaded_scaler.pkl')
        scaler = joblib.load('uploaded_scaler.pkl')
    except Exception as e:
        with output:
            print(f"❌ Error loading scaler: {e}")
        return False

    try:
        save_upload(feature_upload, 'uploaded_features.pkl')
        feature_columns = joblib.load('uploaded_features.pkl')
    except Exception as e:
        with output:
            print(f"❌ Error loading features: {e}")
        return False

    return True

# Data cleaner
def clean_and_process_data(file_obj):
    df = pd.read_csv(file_obj, encoding='ISO-8859-1')
    df = df.drop(df.index[0:2]).reset_index(drop=True)
    mean_idx = df[df['Raw-Pleth#1'] == 'Mean'].index
    drop_idx = set(mean_idx) | set(mean_idx + 1)
    df = df.drop(drop_idx, errors='ignore').reset_index(drop=True)
    df = df.replace({'−': '-'}, regex=True)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.applymap(lambda x: x if str(x).replace('.', '', 1).replace('-', '', 1).isdigit() else None)
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

# Prediction function
def predict_and_filter(df):
    df_clean = df.select_dtypes(include=[float, int]).dropna(axis=1, how='all')
    for col in feature_columns:
        if col not in df_clean.columns:
            df_clean[col] = 0
    df_clean = df_clean[feature_columns]
    X_scaled = scaler.transform(df_clean)
    y_pred = (model.predict(X_scaled) > 0.5).astype(int).flatten()
    df['deleted_flag'] = y_pred
    kept_df = df[df['deleted_flag'] == 0]
    return kept_df

# Button action
def on_run_clicked(b):
    output.clear_output()
    with output:
        if not (model_upload.value and scaler_upload.value and feature_upload.value and data_upload.value):
            print("❗ Please upload all required files before running.")
            return

        print("⏳ Loading model, scaler, and features...")
        if not load_all_files():
            return

        print("✅ Files loaded. Processing data...")

        file_obj = list(data_upload.value.values())[0]['content']
        df = clean_and_process_data(io.BytesIO(file_obj))
        result = predict_and_filter(df)

        print(f"✅ Processing complete. Rows kept: {len(result)}")
        result.to_csv("cleaned_output.csv", index=False)
        display(result.head())
        print("📁 Saved to: cleaned_output.csv")

run_button.on_click(on_run_clicked)

# Display GUI
display(widgets.VBox([
    widgets.HTML("<h3>Upload your model and data</h3>"),
    model_upload,
    scaler_upload,
    feature_upload,
    data_upload,
    run_button,
    output
]))
